In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

print("패키지 로드 완료")

패키지 로드 완료


In [25]:
import json
import time
from datetime import datetime
from kafka import KafkaProducer

# Kafka Producer 설정
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    key_serializer=lambda k: str(k).encode('utf-8') if k else None,
    batch_size=100,
    linger_ms=10,
    acks=1
)

# 이벤트 전송 함수
def send_event(row):
    event = {
        'event_time': datetime.now().isoformat() + ' UTC',
        'event_type': row['event_type'],
        'product_id': str(row['product_id']),
        'category_id': str(row['category_id']),
        'category_code': row['category_code'] if pd.notna(row['category_code']) else None,
        'brand': row['brand'] if pd.notna(row['brand']) else None,
        'price': float(row['price']) if pd.notna(row['price']) else 0.0,
        'user_id': str(row['user_id']),
        'user_session': row['user_session']
    }
    
    producer.send(
        topic='ecommerce-events',
        key=event['user_id'],
        value=event
    )
    return event


In [27]:
# DataFrame 전체 표시 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 전송된 데이터 전체 표시
display(sent_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2025-07-01T14:27:48.468855 UTC,view,44600062,2103807459595387724,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2025-07-01T14:27:48.575729 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2025-07-01T14:27:48.677365 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2025-07-01T14:27:48.783088 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2025-07-01T14:27:48.893244 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2025-07-01T14:27:48.994736 UTC,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
6,2025-07-01T14:27:49.098634 UTC,view,17300353,2053013553853497655,None,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d
7,2025-07-01T14:27:49.201754 UTC,view,31500053,2053013558031024687,None,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6
8,2025-07-01T14:27:49.303922 UTC,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
9,2025-07-01T14:27:49.409557 UTC,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb


In [3]:
# Redis 연결 및 키 확인
import redis
import json
from pprint import pprint

# Redis 클라이언트 생성
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

try:
    r.ping()
    print('Redis 연결 성공')
except Exception as e:
    print(f"Redis 연결 실패: {e}")


Redis 연결 성공


In [4]:
# 1. 모든 user_features 키 조회
user_keys = r.keys("*")
for key in user_keys[:10]:  # 처음 10개만 표시
    print(f"- {key}")
    
if len(user_keys) > 10:
    print(f"... 그 외 {len(user_keys) - 10}개 더 있음")


- category_users:computers.notebook
- user_features:999888777
- user_features:555447699
- category_users:computers.desktop
- user_features:CLEAR_OUTPUT_TEST
- category_users:test.restart
- user_features:RESTART_USER_FIXED
- user_features:537918940
- category_users:furniture.living_room.sofa
- category_users:apparel.shoes.keds
... 그 외 12개 더 있음


In [5]:
# 특정 키의 값 가져오기
value = r.get("user_features:999888777")
print(value)

{"peak_hour":14,"is_early_bird":false,"category_diversity_score":0.0,"user_id":"999888777","total_views_1h":1,"category_counts":{"electronics.smartphone":1},"activity_concentration":1.0,"preferred_time_slot":"afternoon","is_night_owl":false,"timestamp":1751380650194,"top3_categories":["electronics.smartphone"]}
